In [1]:
##import Packages
import pandas as pd
import math
import scipy.stats
from matplotlib import pyplot as plt
import numpy as np

In [2]:
df_Test_Overall = pd.read_csv(r'MTA AB Visit Data Test1_2.csv')
df_rev = pd.read_csv(r'MTA AB Rev Data.csv')
df_cateforymap=pd.read_csv(r'MTA AB Category Mapping.csv')

In [62]:
df_cusid=df_Test_Overall[['SessionID','ControlGroup','CusID','CategoryID','VisitorTypeID']]
df_cusid

,SessionID,ControlGroup,CusID,CategoryID,VisitorTypeID
0,7685274065188263037,1,5299256406894404939,3,4
1,6449284229458809869,1,706543536253348169,1,4
2,509870060026686560,0,2082623938737374415,1,2
3,3589887684654054988,1,7908654078631292723,5,3
4,7081653159511501375,1,8078626604090692368,1,3
...,...,...,...,...,...
13865411,6102357546048967653,0,3245711873504325100,1,4
13865412,4048238120487815131,1,4469163688177765207,3,4
13865413,5709009722269334123,0,772778977274792118,1,4
13865414,9097707239108680272,0,140972094988379001,5,2


In [63]:
df_Rev=pd.merge(df_rev,df_cusid, on = ['SessionID','ControlGroup'])

In [64]:
df_Rev=df_Rev[df_Rev['testid']==2]
df_Rev

,testid,ControlGroup,"""Date""",SessionID,TotalRevenue,CusID,CategoryID,VisitorTypeID
0,2,1,2019-06-26,3170596292771148233,57.98,6955488039014672215,1,4
1,2,1,2019-06-26,4521151331680385048,58.12,6415996569600686642,1,4
2,2,1,2019-06-26,4428930296371807923,383.65,3331805327447778683,1,3
3,2,1,2019-06-26,6052920253472641352,321.28,5259559813374931011,1,3
4,2,1,2019-06-26,2588857339142965515,94.99,8114693136905094140,1,2
...,...,...,...,...,...,...,...,...
1436608,2,1,2019-07-10,4311810967276854497,111.92,8958190399660951713,1,4
1436609,2,1,2019-07-10,8732164761108861961,34.98,8652075078253548853,5,4
1436610,2,0,2019-07-10,5111084495368699663,92.97,7038277058760731937,1,4
1436611,2,1,2019-07-10,3813826744978896205,43.98,1982354719716726752,1,3


In [65]:
df_Rev.drop_duplicates(['ControlGroup','SessionID']).groupby(['ControlGroup'])['SessionID'].count()

ControlGroup
0    386723
1    385706
Name: SessionID, dtype: int64

In [66]:
Control_Rev=df_Rev[df_Rev['ControlGroup']==1]['TotalRevenue'].array
Control_Rev
Var_Rev=df_Rev[df_Rev['ControlGroup']==0]['TotalRevenue'].array
Control_Rev
Var_Rev

<PandasArray>
[ 178.19,   84.49,  331.94,  109.99,   149.9,   55.09,   79.98,   56.99,
  150.95,  869.38,
 ...
  143.11,  138.99,  637.95, 1885.99,   72.99,  111.99, 2395.96,   32.98,
   92.97,   33.98]
Length: 386723, dtype: float64

In [67]:
P1=np.percentile(Control_Rev,95)
P2=np.percentile(Var_Rev,95)

In [68]:
def z_test_calculator_continuous(df,denominator,numerator,numerator_sq):
    #df: dataframe containing the result (absolute counts)
    #denominator: str
    #numerator:str
    #numerator_sq:str
    
    #get the data
    control_denominator=df.loc[1,denominator]
    var_denominator=df.loc[0,denominator]
    control_numerator=df.loc[1,numerator]
    var_numerator=df.loc[0,numerator]    
    
    #Rate
    control_rate=control_numerator/control_denominator
    var_rate=var_numerator/var_denominator
    
    #Variance

    control_var=df.loc[1,numerator_sq]/control_denominator-control_rate**2
    var_var=df.loc[0,numerator_sq]/var_denominator-var_rate**2
    
    #z score
    z_score=(control_rate-var_rate)/math.sqrt(control_var/control_denominator+var_var/var_denominator)
    
    
    #p value
    p_value=scipy.stats.norm.sf(abs(z_score))
    
    
    #lift
    perc_lift=(var_rate-control_rate)/control_rate
    abs_lift=(var_rate-control_rate)
    
    return (p_value,perc_lift,abs_lift)

In [90]:
user_types=['SessionID','CusID']
dic_rev_final={}


for user_type in user_types:
    print(user_type)
    df_Result = pd.DataFrame(df_Rev.drop_duplicates([user_type,'ControlGroup']).groupby('ControlGroup')[user_type].count())
              
    df_Result.loc[1,'Rev']=sum(Control_Rev[Control_Rev<P1])
    df_Result.loc[0,'Rev']=sum(Var_Rev[Var_Rev<P2])

    df_Result.loc[1,'Rev_sq']=sum(Control_Rev[Control_Rev<P1]**2)
    df_Result.loc[0,'Rev_sq']=sum(Var_Rev[Var_Rev<P2]**2)
    
    for index in df_Result.index:
    
        if index!=1:
            df_each_group=df_Result
            df_final=pd.DataFrame()
            result=z_test_calculator_continuous(df_each_group,user_type,'Rev','Rev_sq')

            dic_rev_final[user_type]= result

SessionID
CusID


In [91]:
dic_rev_final['SessionID']

(0.47943916811101006, -0.00010956668021356469, -0.022994679611826996)

In [92]:
dic_rev_final['CusID']

(0.2831296544127835, 0.001119122817330874, 0.26705630482015863)

In [93]:
dic_rev_final

{'SessionID': (0.47943916811101006,
  -0.00010956668021356469,
  -0.022994679611826996),
 'CusID': (0.2831296544127835, 0.001119122817330874, 0.26705630482015863)}

In [96]:
dic_rev_final=pd.DataFrame(dic_rev_final).T
dic_rev_final

,0,1,2
SessionID,0.479439,-0.000110,-0.022995
CusID,0.283130,0.001119,0.267056


In [97]:
dic_rev_final=dic_rev_final.rename(columns={0:'p_value',1:'perc_lift',2:'abs_lift'})
dic_rev_final

,p_value,perc_lift,abs_lift
SessionID,0.479439,-0.000110,-0.022995
CusID,0.283130,0.001119,0.267056


In [98]:
dic_rev_final

,p_value,perc_lift,abs_lift
SessionID,0.479439,-0.000110,-0.022995
CusID,0.283130,0.001119,0.267056


In [99]:
writer = pd.ExcelWriter('Final_revenue_data_Test2.xlsx')
dic_rev_final.to_excel(writer, sheet_name='revenue_data_Test2')
writer.save()

In [76]:
df_rev_data = df_Rev.copy()
dic_rev_final_cuts_usertype = {}
user_types = ['SessionID','CusID']
cuts = ['CategoryID','VisitorTypeID']

In [79]:
for user_type in user_types:
    print(user_type)
    for cut in cuts:
        print(cut)
 
        for p in set(df_rev_data[cut]):

            df_Rev_cut = df_rev_data[df_rev_data[cut]==p]
            Control_Rev_cut=df_Rev_cut[df_Rev_cut['ControlGroup']==1]['TotalRevenue'].array
            Var_Rev_cut=df_Rev_cut[df_Rev_cut['ControlGroup']==0]['TotalRevenue'].array
           
            df_Result_any = pd.DataFrame(df_Rev_cut.drop_duplicates([user_type,'ControlGroup']).groupby('ControlGroup')[user_type].count())
            
            df_Result_any.loc[1,'Rev']=sum(Control_Rev_cut)
            df_Result_any.loc[0,'Rev']=sum(Var_Rev_cut)
            df_Result_any.loc[1,'Rev_sq']=sum(Control_Rev_cut**2)
            df_Result_any.loc[0,'Rev_sq']=sum(Var_Rev_cut**2)

            for index in df_Result.index:

                if index!=1:
                    df_each_group=df_Result_any
                    df_final=pd.DataFrame()
                    result=z_test_calculator_continuous(df_each_group,user_type,'Rev','Rev_sq')

                    dic_rev_final_cuts_usertype [user_type+'_'+cut+str(p)]= result

SessionID
CategoryID
VisitorTypeID
CusID
CategoryID
VisitorTypeID


In [80]:
dic_rev_final_cuts_usertype

{'SessionID_CategoryID1': (0.1669513039029697,
  -0.004007124355840233,
  -1.2190637916609148),
 'SessionID_CategoryID2': (0.13828786389951014,
  -0.012429797836869354,
  -3.87182226136332),
 'SessionID_CategoryID3': (0.18811397157802334,
  -0.012917244256377754,
  -3.67487281400048),
 'SessionID_CategoryID4': (0.04525062502964287,
  -0.0256380939650991,
  -10.138105154206073),
 'SessionID_CategoryID5': (0.08616456625346552,
  0.015975706662562737,
  3.5756524436920927),
 'SessionID_VisitorTypeID1': (0.362676104205879,
  0.005726998795823715,
  1.3826202471863667),
 'SessionID_VisitorTypeID2': (0.13891434473102615,
  0.010722936381297756,
  3.208049489918835),
 'SessionID_VisitorTypeID3': (0.28078399133248366,
  -0.0042619821416727335,
  -1.381876900241764),
 'SessionID_VisitorTypeID4': (0.026697348835632823,
  -0.008650006278191963,
  -2.5486193107296344),
 'CusID_CategoryID1': (0.3264070253001219,
  -0.0018234146574515806,
  -0.6152165643505896),
 'CusID_CategoryID2': (0.084247171920

In [81]:
dic_rev_final_cuts_usertype=pd.DataFrame(dic_rev_final_cuts_usertype).T
dic_rev_final_cuts_usertype

,0,1,2
SessionID_CategoryID1,0.166951,-0.004007,-1.219064
SessionID_CategoryID2,0.138288,-0.012430,-3.871822
SessionID_CategoryID3,0.188114,-0.012917,-3.674873
SessionID_CategoryID4,0.045251,-0.025638,-10.138105
SessionID_CategoryID5,0.086165,0.015976,3.575652
SessionID_VisitorTypeID1,0.362676,0.005727,1.382620
SessionID_VisitorTypeID2,0.138914,0.010723,3.208049
SessionID_VisitorTypeID3,0.280784,-0.004262,-1.381877
SessionID_VisitorTypeID4,0.026697,-0.008650,-2.548619
CusID_CategoryID1,0.326407,-0.001823,-0.615217


In [82]:
dic_rev_final_cuts_usertype=dic_rev_final_cuts_usertype.rename(columns={0:'p_value',1:'perc_lift',2:'abs_lift'})
dic_rev_final_cuts_usertype

,p_value,perc_lift,abs_lift
SessionID_CategoryID1,0.166951,-0.004007,-1.219064
SessionID_CategoryID2,0.138288,-0.012430,-3.871822
SessionID_CategoryID3,0.188114,-0.012917,-3.674873
SessionID_CategoryID4,0.045251,-0.025638,-10.138105
SessionID_CategoryID5,0.086165,0.015976,3.575652
SessionID_VisitorTypeID1,0.362676,0.005727,1.382620
SessionID_VisitorTypeID2,0.138914,0.010723,3.208049
SessionID_VisitorTypeID3,0.280784,-0.004262,-1.381877
SessionID_VisitorTypeID4,0.026697,-0.008650,-2.548619
CusID_CategoryID1,0.326407,-0.001823,-0.615217


In [84]:
writer = pd.ExcelWriter('Final_revenue_cuts_data_Test2.xlsx')
dic_rev_final_cuts_usertype.to_excel(writer, sheet_name='revenue_cuts_data_Test2')
writer.save()